<h1>Applied Data Science Capstone Project</h1>

<h2>Import Packages</h2>

In [36]:
import pandas as pd
import numpy as np
import requests
import geocoder
import folium
from folium.plugins import MarkerCluster

from bs4 import BeautifulSoup

<h2>Scrape the URL for HTML data</h2>

In [37]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text

soup = BeautifulSoup(html_data, "html.parser")

<h2>Preprocess the Table Contents</h2>

In [38]:
columns_titles = ["PostalCode", "Borough", "Neighborhood"]
toronto_neighborhoods = pd.DataFrame(columns = columns_titles)

table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
toronto_neighborhoods = toronto_neighborhoods.append(table_contents)
toronto_neighborhoods['Borough'] = toronto_neighborhoods['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_neighborhoods.shape

(103, 3)

In [39]:
toronto_geodata = pd.read_csv("Geospatial_Coordinates.csv")

toronto_neighborhoods["Postal Code"] = toronto_neighborhoods["PostalCode"]
toronto_neighborhoods = toronto_neighborhoods.drop("PostalCode", axis = 1)

toronto_neighborhoods = pd.merge(toronto_geodata, toronto_neighborhoods, on = "Postal Code")

In [40]:
toronto_neighborhoods.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [41]:
toronto_neighborhoods["PostalCode"] = toronto_neighborhoods["Postal Code"]
toronto_neighborhoods = toronto_neighborhoods.drop("Postal Code", axis = 1)

toronto_neighborhoods = toronto_neighborhoods[["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]]
toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [42]:
toronto_neighborhoods.shape

(103, 5)

<h1>Neighborhood Visualization</h1>

Center a Folium map around the first geospatial location in our data.

In [43]:
m = folium.Map(location = [toronto_neighborhoods["Latitude"][0], toronto_neighborhoods["Longitude"][0]])
m

Show all the neighborhoods on the map using markers.

In [44]:
for i in range(len(toronto_neighborhoods)):
    folium.Marker(
        location = [toronto_neighborhoods.loc[i, "Latitude"], toronto_neighborhoods.loc[i, "Longitude"]],
        popup = toronto_neighborhoods.loc[i, "Neighborhood"],
    ).add_to(m)

m

Cluster our data points using Folium's built in clusterer.

In [45]:
m2 = folium.Map(location = [toronto_neighborhoods["Latitude"][0], toronto_neighborhoods["Longitude"][0]])
marker_cluster = MarkerCluster().add_to(m2)

for i in range(len(toronto_neighborhoods)):
    folium.Marker(
        location = [toronto_neighborhoods.loc[i, "Latitude"], toronto_neighborhoods.loc[i, "Longitude"]],
        popup = toronto_neighborhoods.loc[i, "Neighborhood"],
    ).add_to(marker_cluster)


m2

Zoom in/out to see the clusterer in action!